In [1]:
type ReferenceContigs
    count::Int64
    names::Array{String}
    sizes::Array{Int64}
    offsets::Array{Int64}
    
    function ReferenceContigs(count, names, sizes)
        new(count, names, sizes, [sum(sizes[1:i-1]) for i in 1:length(sizes)])
    end
end

In [2]:
using GZip

import Base: eof, close, position
export BamReader, close, value, eof, advance!, eachposition

type BamReader
    bamStream
    readOrientation #useReverseReads::Bool
    done::Bool
    position::Int64
    chrom::Int32
    contigs::ReferenceContigs
end

function BamReader(bamFileName::String, readOrientation)
    f = GZip.open(bamFileName)

    # make sure this is a BAM file
    code = read(f, UInt8, 4)
    @assert code == b"BAM\1"

    # get through the header data
    l_text = read(f, Int32)
    skip(f, l_text)

    # make sure the contigs match our reference
    n_ref = read(f, Int32)
    sizes = Int32[]
    names = String[]
    for j in 1:n_ref
        l_name = read(f, Int32)
        refName = convert(String, read(f, UInt8, l_name)[1:end-1]) # ignore the null terminator
        l_ref = read(f, Int32)
        push!(sizes, l_ref)
        push!(names, refName)
    end
    
    contigs = ReferenceContigs(n_ref, names, sizes)

    r = BamReader(f, readOrientation, false, 1, -1, contigs)
    advance!(r)
    r
end
close(reader::BamReader) = GZip.close(reader.bamStream)
value(reader::BamReader) = 1
position(reader::BamReader) = reader.position
chrom(reader::BamReader) = get(reader.contigs.names, reader.chrom,"n/a")
eof(reader::BamReader) = reader.position == -1

function advance!(r::BamReader)
    f = r.bamStream
    while !r.done
        if peek(f) == -1 # eof does not work on the BAM files either in C++ or here (BAM vs. gzip issue?)
            r.done = true
            r.position = -1
            return
        end

        buf = Array(Int32, 6) # [block_size, refID, pos, bin_mq_nl, flag_nc]
        gzread(f, pointer(buf), 24)
        block_size = buf[1]
        refID = buf[2] + 1 # the reference contig this read maps to
        lseq = buf[6]

        forward = (buf[5] & 1048576) == 0 # see if we are reverse complemented
        skip(f, block_size-20) # skip the rest of the entry
        
        r.chrom = refID
        #get the position and convert to 1 based indexing
        if forward
            r.position = buf[3] + 1
        else
            r.position = buf[3] + 1 + buf[6]
        end
        
        #work around here
        if get(r.contigs.sizes, refID, 1) < r.position
            r.position = buf[3] + 1
        end

        # break if we found a read in the right direction
        if refID != 0 && (r.readOrientation == :any || (forward && r.readOrientation == :forward) || (!forward && r.readOrientation == :reverse))
            return
        end
    end
end

advance! (generic function with 1 method)

In [3]:
b = BamReader("../data/ATAC1.sorted.bam", :any)

ReferenceContigs(73,String["chr10","chr11","chr12","chr13","chr14","chr15","chr16","chr17_ctg5_hap1","chr17","chr18"  …  "chrUn_gl000242","chrUn_gl000243","chrUn_gl000244","chrUn_gl000245","chrUn_gl000246","chrUn_gl000247","chrUn_gl000248","chrUn_gl000249","chrX","chrY"],[135534747,135006516,133851895,115169878,107349540,102531392,90354753,1680828,81195210,78077248  …  43523,43341,39929,36651,38154,36422,39786,38502,155270560,59373566],[0,135534747,270541263,404393158,519563036,626912576,729443968,819798721,821479549,902674759  …  2919765214,2919808737,2919852078,2919892007,2919928658,2919966812,2920003234,2920043020,2920081522,3075352082])

In [4]:
counts = Dict()
for i in 1:length(c.names)
    counts[c.names[i]] = zeros(Int32, c.sizes[i])
end

In [5]:
b = BamReader("../data/ATAC1.sorted.bam", :any)
cur = "n/a"
while !eof(b)
    try
        counts[chrom(b)][position(b)] += 1
    end
    advance!(b)
end

16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
16572 16571
1657

In [6]:
function writesparse(array, name, dir="./")
    index = Any[]
    values = Any[]
    for i in 1:length(array)
        if array[i] != 0
            push!(values, array[i])
            push!(index, i)
        end
    end
    writecsv("$(dir)$(name)_values.csv", values)
    writecsv("$(dir)$(name)_index.csv", index)
    values, index
end

writesparse (generic function with 2 methods)

In [10]:
for key in keys(counts)
    writesparse(counts[key], key, "ATACdata/")
end